In [190]:
import pandas as pd
import psycopg2
from datetime import datetime, time, timedelta
 

In [191]:
HOST = "rc1a-p8bp15mmxsfwpbt0.mdb.yandexcloud.net"
DB = "db1"
USER = "test_user"
PASSWORD = "j2M{CnnFq@"
PORT = "6432"

Подключение к БД:

In [232]:
try:
    conn = psycopg2.connect(
        host=HOST,
        port=PORT,
        database=DB,
        user=USER,
        password=PASSWORD
    )
    print("Connection successful")
except Exception as e:
    print(f"Error: {e}")   

Connection successful


Вытаскиваем данные из БД и сохраняем в датафреймы

In [ ]:
managers_query = "SELECT * FROM test.managers"
chat_messages_query = "SELECT * FROM test.chat_messages"
rops_query = "SELECT * FROM test.rops"

managers = pd.read_sql_query(managers_query, conn)
chat_messages = pd.read_sql_query(chat_messages_query, conn)
rops = pd.read_sql_query(rops_query, conn)
conn.close()

Переводим в дату поле времени

In [234]:
chat_messages['created'] = pd.to_datetime(chat_messages['created_at'], unit='s')

Ночное время не учитываем. Для этого пересохраняем его в 9:30

In [235]:
def change_time(created):
    if created.hour < 9 or (created.hour == 9 and created.minute < 30):
        return created.replace(hour=9, minute=30, second=0, microsecond=0)
    else:
        return created
chat_messages['created'] = chat_messages['created'].apply(change_time)

In [236]:
chat_messages.head()

,message_id,type,entity_id,created_by,created_at,created
0,"""01jb7da570sf4f65xdf0ptvv9q""",incoming_chat_message,37556493,0,1730046924,2024-10-27 16:35:24
1,"""01jb3t6bc8gvyfnd063nd91ng4""",incoming_chat_message,37549491,0,1729926213,2024-10-26 09:30:00
2,"""01jb6gj9ngwk0ybbmg9w90pbqg""",incoming_chat_message,37531455,0,1730016782,2024-10-27 09:30:00
3,"""01jb75bkprk0d7hht1g5vy1pm4""",incoming_chat_message,37553371,0,1730038583,2024-10-27 14:16:23
4,"""01jb3t5xprbph1c6pym8ervxpm""",incoming_chat_message,37548675,0,1729926199,2024-10-26 09:30:00


Делим сообщения по группам

In [237]:
chat_messages = chat_messages.sort_values(by=['entity_id', 'created'])
chat_messages['group'] = ((chat_messages['entity_id'] != chat_messages.shift()['entity_id']) | (chat_messages['type'] != chat_messages.shift()['type'])).cumsum()

In [238]:
chat_messages = chat_messages.groupby(by='group').first().reset_index()

Ищем разницу в секундах и в днях

In [218]:
chat_messages['time_response'] = chat_messages.groupby('entity_id')['created'].diff() 
chat_messages['days_response'] = chat_messages.groupby('entity_id')['created'].diff().dt.days 
chat_messages = chat_messages.dropna()

Оставляем сообщения менеджеров

In [219]:
messages_response = chat_messages[chat_messages['type'] == 'outgoing_chat_message']

Смотрим,какие ответы были получены не в тот же день

In [230]:
messages_response[messages_response['days_response']!=0]

,group,message_id,type,entity_id,created_by,created_at,created,time_response,days_response,seconds_response
3713,3714,"""01jb7kkdhgf24maz3d3kebrxxw""",outgoing_chat_message,37517839,11490278,1730053518,2024-10-27 18:25:18,1 days 07:41:05,1.0,79865.0
4668,4669,"""01jb7kfvwfdk6qybsytre6xg8a""",outgoing_chat_message,37534855,11448610,1730053402,2024-10-27 18:23:22,1 days 08:53:22,1.0,84202.0
5420,5421,"""01jb6txzatcsv1fx563dz6eebc""",outgoing_chat_message,37537921,11448610,1730027650,2024-10-27 11:14:10,1 days 01:44:10,1.0,58450.0
7880,7881,"""01jb6stjbbawn8zfmdk9dthka3""",outgoing_chat_message,37548901,11448610,1730026490,2024-10-27 10:54:50,1 days 01:24:50,1.0,57290.0
11179,11180,"""01jb766k2gzqr5yhs61bswntf0""",outgoing_chat_message,37551477,11448610,1730039467,2024-10-27 14:31:07,1 days 01:57:49,1.0,59269.0
11568,11569,"""01jb76w6ktarpcma2r69pqmss9""",outgoing_chat_message,37551867,10262513,1730040175,2024-10-27 14:42:55,1 days 01:24:30,1.0,57270.0


Считываем время ответа без учета ночных часов

In [240]:
messages_response.loc[:, 'seconds_response'] = messages_response.apply(
    lambda x: x['time_response'].total_seconds() - x['days_response'] * 60 * 60 * 9.5, axis=1
)
 

Находим среднее время ответов по каждому менеджеру

In [223]:
mean_seconds_work = messages_response.groupby(by='created_by')['seconds_response'].mean().round()

Мерджим с таблицей менеджеров

In [224]:
result = pd.merge(managers, mean_seconds_work, how='inner', left_on = 'mop_id', right_on = 'created_by')


Мерджим с таблицей фирм

In [226]:
result['rop_id'] = result['rop_id'].astype(int)
result = pd.merge(result, rops, how = 'inner')

Получаем окончательный результат

In [227]:
result = result[['name_mop', 'seconds_response', 'rop_name']].sort_values(by='seconds_response')
result.columns = ['Менеджеры', 'Среднее время ответа', 'Фирма']
result

,Менеджеры,Среднее время ответа,Фирма
7,Даша и Даша,219.0,Полина РОП
13,Влада и Настя,283.0,Эля РОП
18,Даша и Карина,348.0,Эля РОП
3,Настя и Даша,354.0,Катя РОП
14,Порхачева Полина,399.0,Эля РОП
19,Полина Мирзоян,559.0,Эля РОП
8,Лиза и Ева,560.0,Полина РОП
21,Ангелина Милованова,614.0,Эля РОП
6,Ксюша и Джамиля,735.0,Катя РОП
12,Настя и Саша,778.0,Полина РОП
